Image augmentation is a technique of altering the existing data to create some more data for the model training process. In other words, it is the process of artificially expanding the available dataset for training a deep learning model

In [35]:
#Importing Modules,Libraries and Packages

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Conv2DTranspose,UpSampling2D,concatenate
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
from os import listdir
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import cv2 
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [36]:
# rotation_range: Int. Degree range for random rotations.
# width_shift_range: Float, 1-D array-like or int(measured as a fraction of the width of the image.)
# height_shift_range: Float, 1-D array-like or int(measured as a fraction of the width of the image.)
# shear_range: Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)
# zoom_range: Float or [lower, upper]. Range for random zoom. If a float, [lower, upper] = [1-zoom_range, 1+zoom_range].
# channel_shift_range: Float. Range for random channel shifts.
# horizontal_flip: Boolean. Randomly flip inputs horizontally.

gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.15, zoom_range=0.1, 
    channel_shift_range=10., horizontal_flip=True)

In [37]:
#Function for plotting the images.

def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

Processing for Augmentation

In [40]:
z=3063
j=3063
for i in range(0,100):

    # Choosing a random image and mask from the directory and setting its dimensions

    chosen_image2 = random.choice(os.listdir('D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\images'))
    image_path1 = 'D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\images/' + chosen_image2
    images = image.load_img(image_path1, target_size=(256,256,3))
    images = image.img_to_array(images)
    images = np.expand_dims(images, axis=0)
    # images.shape



    mask_path1 = 'D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\masks/' + chosen_image2
    mask_path1=mask_path1[:-4]+"_mask.png"
    masks = image.load_img(mask_path1, target_size=(256,256,3))
    masks = image.img_to_array(masks)
    masks = np.expand_dims(masks, axis=0)
    # masks.shape

    
    
    #Next, we'll generate batches of augmented images from the original image.
    #The flow() function takes numpy data and generates batches of augmented data.

    aug_iter = gen.flow(images)
    aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(10)]

    # plotImages(aug_images)


    aug_iter = gen.flow(masks)
    aug_masks = [next(aug_iter)[0].astype(np.uint8) for i in range(10)]

    # plotImages(aug_masks)



    #Saving the images and masks to the directory    
    
    for x, val in zip(gen.flow(images,            #image we chose
        save_to_dir='D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-images',  #Directory to save the data
         save_prefix='x',     # it will save the images as 'x_0_912' some number for every new augmented image
        save_format='png'),range(10)) : # here we define a range because we want 10 augmented images
        pass

    for x, val in zip(gen.flow(masks,             #image we chose
        save_to_dir='D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-masks',   #Directory to save the data
         save_prefix='x',     # it will save the images as 'x_0_912' some number for every new augmented image
        save_format='png'),range(10)) : # here we define a range because we want 10 augmented images otherwise it will keep looping forever I think
        pass

    

    #Renaming the augmentated images and masks saved to the format we require
    
    for i in  os.listdir('D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-images'):
        image_path = 'D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-images/' + i
        new_path='D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-images/' + str(z+1)+".png"
        z+=1
        os.rename(image_path,new_path)

    
    for i in  os.listdir('D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-masks'):
        image_path = 'D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-masks/' + i
        new_path='D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-masks/' + str(j+1)+"_mask.png"
        j+=1
        os.rename(image_path,new_path)



Copying the augmented data to the directory we require

In [42]:
for i in os.listdir('D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-images'):
        shutil.copy('D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-images/' + i, 'D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\images/')

In [43]:
for i in os.listdir('D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-masks'):
        shutil.copy('D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\Augmented-masks/' + i, 'D:\DIC\DIC Brain Tumor\DIC_brain_tumor_dataset\masks/')